In [73]:
from datetime import datetime
import time
import requests
import pandas as pd
import mplfinance as mpf


# 获取比特币K线数据
cycle = '30m'
url = 'https://fapi.binance.com/fapi/v1/klines'
params = {
    'symbol': 'BTCUSDT',
    'interval': cycle,
    'endTime':int(time.time())*1000,
}
res = requests.get(url, params=params)
data = res.json()
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

df = df.apply(pd.to_numeric, errors='ignore')

#save

df['returns'] = (df['close'] - df['open']) / df['open']

df['returns'].to_csv(f'btc_prices_{cycle}.csv')
# 画K线图
#mpf.plot(df, type='candle', volume=True, style='binance')

print(df.tail(10))

                        open     high      low    close     volume  \
timestamp                                                            
2024-04-17 12:00:00  62576.8  62810.9  62301.6  62397.1   8260.846   
2024-04-17 12:30:00  62396.6  62477.0  62024.8  62286.4  12009.840   
2024-04-17 13:00:00  62286.4  62430.2  62100.0  62313.4   6323.703   
2024-04-17 13:30:00  62313.4  62957.2  62204.4  62606.6  13663.403   
2024-04-17 14:00:00  62606.5  62629.8  61025.2  61464.1  35727.868   
2024-04-17 14:30:00  61464.0  61923.4  60663.0  61263.2  32197.185   
2024-04-17 15:00:00  61263.2  61518.6  60368.0  60389.3  28682.541   
2024-04-17 15:30:00  60389.3  60701.6  59713.2  60272.0  43337.208   
2024-04-17 16:00:00  60274.0  61086.2  59625.0  60714.7  36078.119   
2024-04-17 16:30:00  60714.7  60743.3  60351.1  60430.8   3285.964   

                        close_time  quote_asset_volume  number_of_trades  \
timestamp                                                                  
2024-04

In [80]:
import requests
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta

def rolling_get_btc_lob_factor(cycle_:int):
    url = 'https://fapi.binance.com/fapi/v1/depth'
    params = {
        'symbol': 'BTCUSDT',
        'interval': cycle_,
    }
    # 初始化列表来存储数据
    data = []

    # 用于存储bid-ask差异值的列表
    bid_ask_diffs = []

    def fetch_data():
        response = requests.get(url, params=params)
        depth_data = response.json()
        bids = depth_data['bids'][:1000]
        asks = depth_data['asks'][:1000]
        
        # 计算bid-ask差异值并添加到列表
        bid_ask_diffs.append(sum((float(bid[1]) - float(ask[1])) for bid, ask in zip(bids, asks)))

    # 每分钟执行一次fetch_data
    cycle = cycle_
    for _ in range(cycle):  # 假设我们想要持续30分钟
        fetch_data()
        time.sleep(60)  # 等待60秒

    # 计算最近30分钟的均值
    if len(bid_ask_diffs) >= cycle:
        mean_diff = np.mean(bid_ask_diffs[-cycle:])
        print(f"最近30分钟的均值: {mean_diff}")

    # 获取当前的日期和时间
    now = datetime.now()

    # 四舍五入到最近的30分钟
    # 先计算分钟数
    minute = now.minute
    # 如果分钟数小于30，则四舍五入到0分钟，否则四舍五入到下一个30分钟（即59分钟）
    if minute < 30:
        rounded_time = now.replace(minute=0, second=0, microsecond=0)
    else:
        rounded_time = now.replace(minute=30, second=0, microsecond=0)

    # 格式化日期时间
    formatted_time = rounded_time.strftime('%Y-%m-%d %H:%M:%S')

    # 使用pandas计算zscore
    bid_ask_diffs_serise = pd.Series(bid_ask_diffs)
    zscore = ( bid_ask_diffs_serise- bid_ask_diffs_serise.mean()) / bid_ask_diffs_serise.std()
    df = pd.DataFrame({
        'timestamp': formatted_time,
        'zscore' : zscore
    })


    # 将数据存入csv文件
    if pd.read_csv('bid_ask_diffs_factor.csv').shape[0] == 0:
        df.to_csv('bid_ask_diffs_factor.csv', index=False)
    else:
        df.to_csv('bid_ask_diffs_factor.csv', index=False,mode = 'a',header=False)
    # 将数据存入csv 模型训练文件
    df_raw = pd.read_csv('btc_prices_30m.csv')
    if formatted_time in df['timestamp']:
        # 如果存在，找到对应的行索引
        row_index = df.index[df['timestamp'] == formatted_time][0]
        # 然后为特定的列赋值
        df.at[row_index, 'lob_factor'] = zscore
    print('lob因子写入csv了')

#30m采集一次
while True:
    rolling_get_btc_lob_factor(30)

TypeError: list indices must be integers or slices, not str

In [ ]:
#train model 
